In [55]:
import pandas as pd, tarfile

Since the data is formatted in different ways, we create a file that unifies the environment data in this notebook.

### 1. Skip first hour

As illustrated below, the solar irradiance and rate consumption datasets starts at hour 1, not at 0, as other datasets do. Thus, we think that it makes sense to skip the first hour in all other datasets to start at the same time.

In [56]:
# This dataset (and rate consumption) starts at hour 1, ...
solar_irradiance = pd.read_csv("../data/SolarIrradiance.csv")[["Year", "HOUR-PST", "Avg Global Horizontal [W/m^2]"]]
solar_irradiance.head(2)

,Year,HOUR-PST,Avg Global Horizontal [W/m^2]
0,2016,1,0.1592
1,2016,2,0.0850


In [57]:
# ... but has hour 0 somewhere appearing in the middle of it
solar_irradiance.iloc[22:25]

,Year,HOUR-PST,Avg Global Horizontal [W/m^2]
22,2016,23,0.0742
23,2016,0,0.0417
24,2016,1,0.0000


In [58]:
# However, this dataset (and load data) start at hour 0, and ...
wind_speed = pd.read_csv("../data/WindSpeed.csv")[["Year","Hour","Wind Speed  "]]
wind_speed["Wind Speed  "] = wind_speed["Wind Speed  "].apply(lambda mps: 3.6 * mps) # convert m/s to km/h, like in the given loading code
wind_speed.head(2)

,Year,Hour,Wind Speed
0,2016.0,0.0,43.128
1,2016.0,1.0,45.900


In [59]:
# also contain an hour 0 for following days.
wind_speed.iloc[23:26]

,Year,Hour,Wind Speed
23,2016.0,23.0,23.904
24,2016.0,0.0,18.432
25,2016.0,1.0,14.040


Therefore, we skip the first line in all datasets other than the solar irradiance datasets.

In [60]:
rate_cons = pd.read_csv("../data/rate_consumption_charge.csv")[['Year', 'HOUR-PST', 'Grid Elecricity Price锛?/kWh锛?2016',]]
rate_cons = rate_cons.rename(columns={'Grid Elecricity Price锛?/kWh锛?2016': "Grid Electricity Price"})
rate_cons.head(5)

,Year,HOUR-PST,Grid Electricity Price
0,2016,1,0.06
1,2016,2,0.06
2,2016,3,0.06
3,2016,4,0.06
4,2016,5,0.06


In [61]:
households = {}
with tarfile.open("../data/COMMERCIAL_LOAD_DATA_E_PLUS_OUTPUT.part1.tar.gz") as tf:
    for name, archive_path in [
        ("warehouse 1", "USA_CA_Palmdale.AP.723820_TMY3/RefBldgWarehouseNew2004_7.1_5.0_3B_USA_CA_LOS_ANGELES.csv"),
        ("small hotel 1", "USA_AR_Little.Rock.AFB.723405_TMY3/RefBldgSmallHotelNew2004_v1.3_7.1_3A_USA_GA_ATLANTA.csv"),
    ]:
        file = tf.extractfile(archive_path)
        households[name] = pd.read_csv(file)[['Date/Time', 'Electricity:Facility [kW](Hourly)']]

In [62]:
households["warehouse 1"].head(3)

,Date/Time,Electricity:Facility [kW](Hourly)
0,01/01 01:00:00,16.895495
1,01/01 02:00:00,16.696428
2,01/01 03:00:00,17.220737


In [63]:
households["small hotel 1"].head(3)

,Date/Time,Electricity:Facility [kW](Hourly)
0,01/01 01:00:00,41.874827
1,01/01 02:00:00,41.094081
2,01/01 03:00:00,35.364626


In [64]:
# skip the first hour in every dataset except solar_irradiance such that they fit each other and the beginnings are synced
wind_speed = wind_speed.iloc[1:].reset_index(drop=True)
households = {name: household.iloc[1:].reset_index(drop=True) for name, household in households.items()}


### 2. Cap data

Furthermore, the different datasets are of different lengths. Therefore, we pick the length of the shortest dataset to have features of equal lengths.

Note that there is only data for half a year available for most datasets, whereas for the load data we have a full year available.

In [65]:
rate_cons.head(5)

,Year,HOUR-PST,Grid Electricity Price
0,2016,1,0.06
1,2016,2,0.06
2,2016,3,0.06
3,2016,4,0.06
4,2016,5,0.06


In [67]:
dataset_lengths = (len(wind_speed), len(solar_irradiance), len(rate_cons), *[len(households[name]) for name in households])
dataset_lengths

(4321, 4319, 4319, 8759, 8759)

In [68]:
min_length = min(dataset_lengths); min_length

4319

In [69]:
wind_speed = wind_speed.iloc[:min_length]
solar_irradiance = solar_irradiance.iloc[:min_length]
rate_cons = rate_cons.iloc[:min_length]
households = {name: household.iloc[:min_length] for name, household in households.items()}

In [70]:
len(wind_speed), len(solar_irradiance), len(rate_cons), *[len(household) for household in households.values()]

(4319, 4319, 4319, 4319, 4319)

### Export

In [79]:
df = pd.DataFrame({
  "Wind Speed": wind_speed["Wind Speed  "],
  "Solar Irradiance": solar_irradiance["Avg Global Horizontal [W/m^2]"],
  "Grid Electricity Price": rate_cons["Grid Electricity Price"],
  **{name: household["Electricity:Facility [kW](Hourly)"] for name, household in households.items()},
});
df

,Wind Speed,Solar Irradiance,Grid Electricity Price,warehouse 1,small hotel 1
0,45.900,0.1592,0.06,16.696428,41.094081
1,46.872,0.0850,0.06,17.220737,35.364626
2,47.340,0.0000,0.06,16.837994,34.643026
3,40.608,0.0000,0.06,17.380582,35.939681
4,42.732,0.0000,0.06,16.934718,40.016787
...,...,...,...,...,...
4314,6.480,67.7188,0.19,15.415824,134.813247
4315,9.036,0.9450,0.19,15.415824,127.089200
4316,9.828,0.0695,0.06,15.415824,96.408565
4317,2.592,0.1174,0.06,15.415824,69.564282


In [80]:
df.to_csv("../data/cropped.csv", index=False)